In [ ]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from bs4 import BeautifulSoup
import re
from time import sleep
import pickle

In [ ]:
def search(driver, city, state):
    textbox = driver.find_element_by_xpath('//input[@class="large has-menu"]')
    textbox.click()
    textbox.send_keys(str(city)+', '+str(state), Keys.ENTER)

In [3]:
def get_links_from_page(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source)
    links = soup.findAll('a', {'class': 'result-name'})
    hrefs = []
    for l in links:
        string = str(l)
        entry = re.findall('(?<=href=\")(.*)(?=\" item)', string)
        hrefs.append(entry[0])
    return hrefs

In [4]:
def next_page(driver):
    nextpage=driver.find_element_by_xpath('//a[@class="btn btn-default btn-next"]')
    if nextpage:
        print('Moving to next page ...')
        nextpage.click()
    else:
        print('No more pages ...')

In [5]:
def pagination(driver):
    link_text=get_links_from_page(driver)
    next_page(driver)
    return link_text

In [74]:
def collect_links():
    results=[]
    chrome_option=ChromeOptions()
    chrome_option.add_argument('--headless')
    driver=Chrome(r'C:\Users\17247\Desktop\Werk\ldapa\driver\chromedriver.exe', options=chrome_option)
    driver.get('https://www.psychologytoday.com/us')
    search(driver, 'Pittsburgh', 'PA')
    while True:
        try:
            entry=pagination(driver)
            results.append(entry)
        except StaleElementReferenceException as e:
            print('Error:', e)
            continue
        except NoSuchElementException as e:
            print('No more pages ...')
            break
    driver.close()
    return results

In [88]:
def parse_pages(list_of_links):
    result_list=[]
    driver=Chrome(r'C:\Users\17247\Desktop\Werk\ldapa\driver\chromedriver.exe')
    for l in list_of_links:
        result={}
        driver.get(l)
        soup=BeautifulSoup(driver.page_source)
        name=soup.find('h1', {'itemprop':'name'})
        if name:
            result['name']=name.text
        else:
            result['name']='None'
        phone=soup.find('a',{'data-event-label':'Profile_PhoneLink'})
        if phone:
            result['phone']=phone.text
        else:
            result['phone']='None'
        city=soup.find('span',{'itemprop':'addressLocality'})
        if city:
            result['city']=city.text.replace(',','')
        else:
            result['city']='None'
        state=soup.find('span',{'itemprop':'addressRegion'})
        if state:
            result['state']=state.text
        else:
            result['state']='None'
        zip_code=soup.find('span',{'itemprop':'postalcode'})
        if zip_code:
            result['zip_code']=zip_code.text
        else:
            result['zip_code']='None'
        specalites=soup.findAll('li', {'class':'highlight'})
        if specalites:
            spec_text = [s.text.strip() for s in specalites]
            result['specs']=spec_text
        else:
            result['specs']='None'
        result_list.append(result)
    return result_list

In [7]:
r=collect_links()

Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Error: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=79.0.3945.117)

Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Error: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=79.0.3945.117)

Moving to next page ...
Moving to next page ...
Moving to next page ...
Moving to next page ...
Error: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=79.0.3945.117)

Moving to next page ...
Moving to next page

In [89]:
final_list = []
for sublist in r:
    for item in sublist:
        final_list.append(item)

In [93]:
list_to_parse=list(set(final_list))

In [94]:
data_to_enter=parse_pages(list_to_parse)

In [95]:
len(data_to_enter)

640

In [100]:
with open("pitt_data.txt", "wb") as fp:
    pickle.dump(data_to_enter, fp)